    At the beginning we wanted to know what is the distance between source and target when we removed the link between both but I did not find a way to compute that easily 

In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

In [2]:
from core.io.loader import get_data_with_node_information


In [3]:
import igraph

In [4]:
import pandas as pd

In [5]:
from typing import Tuple, Dict

In [6]:
import numpy as np

In [58]:
train = get_data_with_node_information('train', data_dir='../data')


In [8]:
connected_train = train[train.category == 1]

In [14]:
nodes_mapping = {value:key for key, value in train.source_id.drop_duplicates().reset_index(drop=True).to_dict().items()}

In [15]:
edges = []
for source_id, target_id in zip(connected_train.source_id, connected_train.target_id):
    edges.append((nodes_mapping[source_id], nodes_mapping[target_id]))

In [16]:
graph = igraph.Graph(edges)

In [17]:
graph.vcount()

27770

In [18]:
graph.ecount()

335130

In [19]:
def create_paper_graph(df: pd.DataFrame) -> Tuple[igraph.Graph, Dict]:
    nodes_mapping = df.source_id.drop_duplicates().reset_index(drop=True).to_dict()
    nodes_mapping = {value:key for key, value in nodes_mapping.items()}
    edges = []
    connected_papers = df[df.category == 1]
    for source_id, target_id in zip(connected_papers.source_id, connected_papers.target_id):
        edges.append((nodes_mapping[source_id], nodes_mapping[target_id]))
    graph = igraph.Graph(edges)
    return graph, nodes_mapping

In [20]:
graph, nodes_mapping

# Features

In [21]:
from features.collaboration import extract_distances_series, extract_distances_matrix

In [22]:
import time

In [23]:
neighbors = {}
for i in range(graph.vcount()):
    neighbors[i] = set(graph.neighbors(i))

In [24]:
common_neighbors = []
for source_id, target_id in zip(train.source_id.values, train.target_id.values):
    source_id = nodes_mapping[source_id]
    target_id = nodes_mapping[target_id]
    common_neighbors.append(len(neighbors[source_id].intersection(neighbors[target_id])))

In [59]:
train['source_target_common_neighbors'] = pd.Series(common_neighbors, index=train.index)

In [62]:
paper_citations = train[train.category == 1].target_id.value_counts().rename('paper_citations')
train = train.merge(paper_citations.to_frame(), how='left', right_index=True, left_on='source_id')


train = train.merge(paper_citations.to_frame(),
                    how='left', right_index=True, left_on='target_id',
                    suffixes = ('_source', '_target'))


train = train.rename(columns={'paper_citations_source': 'source_paper_citations',
                      'paper_citations_target': 'target_paper_citations'})

In [63]:
number_of_papers_cited = train[train.category == 1].source_id.value_counts().rename('number_of_papers_cited')
train = train.merge(number_of_papers_cited.to_frame(), how='left', right_index=True, left_on='source_id')


train = train.merge(number_of_papers_cited.to_frame(),
                    how='left', right_index=True, left_on='target_id',
                    suffixes = ('_source', '_target'))

train = train.rename(columns={'number_of_papers_cited_source': 'source_number_of_papers_cited',
                      'number_of_papers_cited_target': 'target_number_of_papers_cited'})
